Deep Learning
=============

Assignment 2.1
------------

在之前的任务中，我们创建了一个经过处理后的数据集，在这个任务中，我们会基于这个数据集用Tensorflow逐渐训练一个越来越深越来越精确的模型。

In [9]:
# 声明库依赖
import pickle
import numpy as np
import tensorflow as tf

image_size = 28
num_labels = 10

In [10]:
class NotMNIST:
    def __init__(self):
        class Train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0

            @property
            def num_examples(self):
                return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter + num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter + num]
                    self.batch_counter += num
                return batch_images, batch_labels

        class Test:
            def __init__(self):
                self.images = []
                self.labels = []
        
        class Validation:
            def __init__(self):
                self.images = []
                self.labels = []

        self.train = Train()
        self.test = Test()
        self.validation = Validation()

        pickle_file = '../data/notMNIST.pickle'

        with open(pickle_file, 'rb') as f:
            save = pickle.load(f)
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            valid_dataset = save['valid_dataset']
            valid_labels = save['valid_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
            del save  # 删除内存文件，等待gc回收释放内存

        def reformat(dataset, labels):
            dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
            labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
            return dataset, labels

        train_dataset, train_labels = reformat(train_dataset, train_labels)
        valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
        test_dataset, test_labels = reformat(test_dataset, test_labels)
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)
        self.train.images = train_dataset
        self.train.labels = train_labels
        self.test.images = test_dataset
        self.test.labels = test_labels
        self.validation.images = valid_dataset
        self.validation.labels = valid_labels

In [14]:
not_mnist = NotMNIST()

# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))


################################################
# 添加一个拥有256个神经元的隐藏层
################################################


# 构造模型
pred = tf.nn.softmax(tf.matmul(x, W) + b)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))

# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()


# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.811057055
Epoch: 0002 cost= 0.681903819
Epoch: 0003 cost= 0.663407988
Epoch: 0004 cost= 0.653774098
Epoch: 0005 cost= 0.647646352
Epoch: 0006 cost= 0.643349932
Epoch: 0007 cost= 0.640144937
Epoch: 0008 cost= 0.637645058
Epoch: 0009 cost= 0.635627633
Epoch: 0010 cost= 0.633955275
Epoch: 0011 cost= 0.632538748
Epoch: 0012 cost= 0.631317642
Epoch: 0013 cost= 0.630249594
Epoch: 0014 cost= 0.629304015
Epoch: 0015 cost= 0.628458234
Epoch: 0016 cost= 0.627695059
Epoch: 0017 cost= 0.627001206
Epoch: 0018 cost= 0.626366215
Epoch: 0019 cost= 0.625781743
Epoch: 0020 cost= 0.625241022
Epoch: 0021 cost= 0.624738514
Epoch: 0022 cost= 0.624269631
Epoch: 0023 cost= 0.623830523
Epoch: 0024 cost= 0.623417954
Epoch: 0025 cost= 0.623029157
Optimization Finished!
Accuracy: 0.8989


In [ ]:
################################################
# 将神经元的数目改为512
################################################